# Gerador de dados
Script focada na geração de dados artificiais para o projeto Enterprise Challenge - FIAP

## Preparações iniciais

In [2]:
import pandas as pd, random
import numpy as np
from faker import Faker
import datetime

### Definindo parametros iniciais

In [3]:
id_maquina = 101
id_potenciometro = 1
id_termistor = 2
id_mpu6050 = 3

reg_max = 50000


## Setando valores padrões

In [4]:

# Inicializa o Faker para gerar dados em português brasileiro
fake = Faker('pt_BR')

# Para garantir que os dados "aleatórios" sejam sempre os mesmos, o que é bom para testes
Faker.seed(0)
np.random.seed(0)


In [41]:

def gerar_fabricas(num_fabricas):
    print("Iniciando a geração de dados para as Dimensões...")
    fabricas = []
    for i in range(1, num_fabricas + 1):
        fabricas.append({
            'id_fabrica': i,
            'nome_fabrica': fake.company(),
            'cidade': fake.city(),
            'estado': fake.state_abbr()
        })
    df_fabricas = pd.DataFrame(fabricas)
    return(df_fabricas)


In [ ]:
def gerar_sensores(num_sensores_gerados):
    # --- Dimensão Sensores ---
    tipos_sensor = {'Temperatura':'Ds18b20', 'Vibracao':'MPU-6050', 'Corrente':'3590S'}
    sensores = []
    id_sensor_counter = 1
    for tipo in tipos_sensor:
        for i in range(0, num_sensores_gerados):
            
            sensores.append({
                'id_sensor': id_sensor_counter,
                'nome_sensor': f'{tipos_sensor[tipo]}-{fake.bothify(text="???-####")}',
                'tipo_sensor': tipo,
                'fabricante_sensor': fake.company(),
                'faixa_operacional_min': '',
                'faixa_operacional_max': ''
            })
            id_sensor_counter += 1

    df_sensores = pd.DataFrame(sensores)
    df_sensores.loc[df_sensores['tipo_sensor'] == 'Temperatura', 'faixa_operacional_min'] = -55.0
    df_sensores.loc[df_sensores['tipo_sensor'] == 'Temperatura', 'faixa_operacional_max'] = 125.0
    df_sensores.loc[df_sensores['tipo_sensor'] == 'Vibracao', 'faixa_operacional_min'] = 0.0
    df_sensores.loc[df_sensores['tipo_sensor'] == 'Vibracao', 'faixa_operacional_max'] = 150.00
    df_sensores.loc[df_sensores['tipo_sensor'] == 'Corrente', 'faixa_operacional_min'] = 0.0
    df_sensores.loc[df_sensores['tipo_sensor'] == 'Corrente', 'faixa_operacional_max'] = 0.8
    return df_sensores


In [43]:
def gerar_lote(qtd_lote, frequencia, minutos_lote, data_inicial: str = "2025-08-01 08:00"):
    '''
    qtd_lote: Quantos lotes serão gerados
    frequencia: A frequencia de geração de dados de cada lote
    minutos_lote: Quantos minutos serão considerados 1 lote inteiro
    '''
    periodo = (60/frequencia)*minutos_lote
    print(periodo)
    print(int(data_inicial.split('-')[0]),int(data_inicial.split('-')[1]),int(data_inicial.split('-')[2].split(' ')[0]),int(data_inicial.split('-')[2].split(' ')[1].split(':')[0]))

    data_inicial = datetime.datetime(
        int(data_inicial.split('-')[0]),
        int(data_inicial.split('-')[1]),
        int(data_inicial.split('-')[2].split(' ')[0]),
        int(data_inicial.split('-')[2].split(' ')[1].split(':')[0]),
        int(data_inicial.split('-')[2].split(' ')[1].split(':')[1]),
        0,
        0
        )
    
    df_lote = pd.DataFrame(
        {
        'id':[],
        'timestamp':[]
        }
    )

    for a in range(1, qtd_lote+1):
        if a == 1:
            lote_temp = pd.DataFrame({'timestamp':pd.date_range(data_inicial, periods=periodo, freq=f'{int(frequencia)}min')}) 
        else:
            lote_temp = pd.DataFrame({'timestamp':pd.date_range(max(df_lote['timestamp']), periods=periodo, freq=f'{int(frequencia)}min')})
        lote_temp['id'] = a
        df_lote = pd.concat([df_lote, lote_temp[['id', 'timestamp']]], ignore_index=True)

    df_lote = df_lote.astype({'id':int})
    return df_lote


In [44]:
def gerar_maquinas(num_maquinas):
    # --- Dimensão Máquinas ---
    maquinas = []
    ids_fabricas_validos = df_fabricas['id_fabrica'].tolist()
    for i in range(1, num_maquinas + 1):
        maquinas.append({
            'id_maquina': i,
            'nome_maquina': f'WeldBot-{2000 + i}',
            'fabricante': fake.company(),
            'modelo': np.random.choice(['TX-100', 'RX-500', 'W-Master']),
            'dt_instalacao': fake.past_date(start_date='-2y'),
            'id_fabrica': np.random.choice(ids_fabricas_validos) # Garante integridade
        })
    df_maquinas = pd.DataFrame(maquinas)
    return df_maquinas


In [45]:
# print(int(len(df_sensores['id_sensor'])/3))
def gerar_sistema(qtd_sistemas):
    sistema = []

    for i in range(1, qtd_sistemas + 1):
        sistema.append({
            'id_sistema':i,
            'endereco_sistema':fake.bothify(text="SETOR-####"),
            'nome_sistema':f'ESP32-',
            # 'id_maquina':random.choice(df_maquinas['id_maquina'].tolist())
        })

    df_sistema = pd.DataFrame(sistema)
    df_sistema['nome_sistema'] = df_sistema['nome_sistema'] + df_sistema['endereco_sistema']
    return df_sistema

# df_sistema = gerar_sistema()
# # df_sistema['id_maquina'] = random.choice(df_maquinas['id_maquina'].tolist())
# display(df_sistema)

In [118]:
df_fabricas = gerar_fabricas(6)
display(df_fabricas)


Iniciando a geração de dados para as Dimensões...


,id_fabrica,nome_fabrica,cidade,estado
0,1,Moura,Lopes de Costa,MG
1,2,Fogaça e Filhos,Ribeiro dos Dourados,RJ
2,3,Nunes,Peixoto Verde,AM
3,4,da Paz Cavalcante e Filhos,da Cunha Grande,RS
4,5,das Neves - ME,Sá das Pedras,RN
5,6,Monteiro S.A.,Vieira do Oeste,AC


In [ ]:
df_sensores = gerar_sensores(4)
display(df_sensores)


,id_sensor,nome_sensor,tipo_sensor,fabricante_sensor,faixa_operacional_min,faixa_operacional_max,id_maquina
0,1,Ds18b20-Ync-3524,Temperatura,Jesus,-55.0,125.0,1
1,2,Ds18b20-sfi-8532,Temperatura,Farias S/A,-55.0,125.0,2
2,3,Ds18b20-hUd-2181,Temperatura,Casa Grande - ME,-55.0,125.0,3
3,4,Ds18b20-gmM-9407,Temperatura,da Conceição S.A.,-55.0,125.0,4
4,5,MPU-6050-sMM-0974,Vibracao,Moraes Peixoto - ME,0.0,150.0,1
5,6,MPU-6050-JjE-5121,Vibracao,Albuquerque Pereira S/A,0.0,150.0,2
6,7,MPU-6050-qCF-4314,Vibracao,da Mata Rocha - EI,0.0,150.0,3
7,8,MPU-6050-CoM-8674,Vibracao,Caldeira,0.0,150.0,4
8,9,3590S-VxO-6631,Corrente,Ribeiro Sousa S/A,4,8,1
9,10,3590S-wmT-8106,Corrente,Farias Lopes S/A,4,8,2


In [ ]:
df_sistema = gerar_sistema(int(len(df_sensores['id_sensor'])/3))
df_sensores.loc[df_sensores['tipo_sensor'] == "Temperatura", "id_maquina"] = [int(a) for a in df_sistema['id_sistema']]
df_sensores.loc[df_sensores['tipo_sensor'] == "Vibracao", "id_maquina"] = [int(a) for a in df_sistema['id_sistema']]
df_sensores.loc[df_sensores['tipo_sensor'] == "Corrente", "id_maquina"] = [int(a) for a in df_sistema['id_sistema']]
df_sensores['id_maquina'] = df_sensores['id_maquina'].astype(int) 
display(df_sistema)

,id_sistema,endereco_sistema,nome_sistema
0,1,SETOR-8857,ESP32-SETOR-8857
1,2,SETOR-1688,ESP32-SETOR-1688
2,3,SETOR-6119,ESP32-SETOR-6119
3,4,SETOR-7524,ESP32-SETOR-7524


In [129]:
df_lote = gerar_lote(5, 60, 1)
display(df_lote)


1.0
2025 8 1 8


C:\Users\Davi\AppData\Local\Temp\ipykernel_15860\136447155.py:30: FutureWarning: Non-integer 'periods' in pd.date_range, pd.timedelta_range, pd.period_range, and pd.interval_range are deprecated and will raise in a future version.
  lote_temp = pd.DataFrame({'timestamp':pd.date_range(data_inicial, periods=periodo, freq=f'{int(frequencia)}min')})
C:\Users\Davi\AppData\Local\Temp\ipykernel_15860\136447155.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_lote = pd.concat([df_lote, lote_temp[['id', 'timestamp']]], ignore_index=True)
C:\Users\Davi\AppData\Local\Temp\ipykernel_15860\136447155.py:32: FutureWarning: Non-integer 'periods' in pd.date_range, pd.timedelta_range, pd.period_range, and pd.interval_range are deprecated and will rais

,id,timestamp
0,1,2025-08-01 08:00:00
1,2,2025-08-01 08:00:00
2,3,2025-08-01 08:00:00
3,4,2025-08-01 08:00:00
4,5,2025-08-01 08:00:00


In [128]:
df_maquinas = gerar_maquinas(len(df_fabricas))
display(df_maquinas)


,id_maquina,nome_maquina,fabricante,modelo,dt_instalacao,id_fabrica
0,1,WeldBot-2001,Barbosa S.A.,TX-100,2025-03-14,6
1,2,WeldBot-2002,Macedo,W-Master,2023-11-19,1
2,3,WeldBot-2003,Ramos,RX-500,2023-10-14,3
3,4,WeldBot-2004,Casa Grande Vargas e Filhos,TX-100,2024-08-11,3
4,5,WeldBot-2005,Cavalcante,TX-100,2025-09-19,6
5,6,WeldBot-2006,Castro,W-Master,2024-04-02,3


In [134]:
def gerar_fatos(maquinario, sensores, lotes, reg: int = 5000):
    print("\nIniciando a geração de dados para as Tabelas Fato...")

    # Extrai listas de IDs válidos das dimensões
    ids_maquinas_validos = maquinario['id_maquina'].tolist()
    ids_sensores_validos = sensores['id_sensor'].tolist()
    ids_lotes_validos = lotes['id'].tolist()
    print(ids_maquinas_validos, ids_sensores_validos, ids_lotes_validos)
    # # --- Fato Registros de Sensores ---
    registros = {
        'id_registro': range(1, reg + 1),
        'id_sensor': np.random.choice(ids_sensores_validos, reg),
        'id_maquina': np.random.choice(ids_maquinas_validos, reg),
        'id_lote': np.random.choice(ids_lotes_validos, reg),
        'timestamp_registro': pd.to_datetime(np.linspace(
            datetime.datetime(2025, 8, 1).timestamp(),
            datetime.datetime.now().timestamp(),
            reg), unit='s'),
        'vl_temperatura': np.random.normal(80, 15, reg),
        'vl_vibracao': np.random.normal(4, 2, reg),
        'vl_corrente': np.random.normal(150, 5, reg)
    }
    df_registros = pd.DataFrame(registros)

    # df_registros = pd.DataFrame(registros)
    return df_registros
    # display(df_registros)
df_fato = gerar_fatos(df_maquinas, df_sensores, df_lote)
df_fato.to_excel(r'C:\Users\Davi\Documents\Projetos\FIAP\FASE 6\Enterprise Challenge Fase 4\docs\estrutura de teste\Dados_simulados.xlsx', index=False)


Iniciando a geração de dados para as Tabelas Fato...
[1, 2, 3, 4, 5, 6] [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] [1, 2, 3, 4, 5]
